In [2]:
!pip install ultralytics

# !pip install -q pyyaml


In [3]:
import os
import shutil

# Thư mục làm việc trên Kaggle
working_dir = '/kaggle/working'
os.chdir(working_dir)

# Đường dẫn dữ liệu đầu vào
input_data_dir = '/kaggle/input/drown-datasets/datasets'
train_dir = os.path.join(input_data_dir, 'train')
val_dir = os.path.join(input_data_dir, 'val')

# Thư mục đích trong /kaggle/working
output_train_images = os.path.join(working_dir, 'datasets', 'train', 'images')
output_train_labels = os.path.join(working_dir, 'datasets', 'train', 'labels')
output_val_images = os.path.join(working_dir, 'datasets', 'val', 'images')
output_val_labels = os.path.join(working_dir, 'datasets', 'val', 'labels')

# Tạo thư mục đích
os.makedirs(output_train_images, exist_ok=True)
os.makedirs(output_train_labels, exist_ok=True)
os.makedirs(output_val_images, exist_ok=True)
os.makedirs(output_val_labels, exist_ok=True)

# Hàm sao chép và phân loại dữ liệu
def prepare_data(src_dir, img_dst_dir, lbl_dst_dir):
    if os.path.exists(os.path.join(src_dir, 'images')) and os.path.exists(os.path.join(src_dir, 'labels')):
        # Trường hợp 1: Đã có images/ và labels/
        shutil.copytree(os.path.join(src_dir, 'images'), img_dst_dir, dirs_exist_ok=True)
        shutil.copytree(os.path.join(src_dir, 'labels'), lbl_dst_dir, dirs_exist_ok=True)
    else:
        # Trường hợp 2: Ảnh và nhãn lẫn lộn
        for file in os.listdir(src_dir):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                shutil.copy(os.path.join(src_dir, file), img_dst_dir)
            elif file.endswith('.txt'):
                shutil.copy(os.path.join(src_dir, file), lbl_dst_dir)

# Chuẩn bị dữ liệu
print("Chuẩn bị dữ liệu train...")
prepare_data(train_dir, output_train_images, output_train_labels)
print("Chuẩn bị dữ liệu val...")
prepare_data(val_dir, output_val_images, output_val_labels)

# Kiểm tra dữ liệu
for path, name in [(output_train_images, 'train images'), (output_train_labels, 'train labels'),
                   (output_val_images, 'val images'), (output_val_labels, 'val labels')]:
    exists = os.path.exists(path)
    print(f"Kiểm tra thư mục {name}: {exists}")
    if exists:
        print(f"Số lượng file trong {name}: {len(os.listdir(path))}")

Chuẩn bị dữ liệu train...
Chuẩn bị dữ liệu val...
Kiểm tra thư mục train images: True
Số lượng file trong train images: 7000
Kiểm tra thư mục train labels: True
Số lượng file trong train labels: 7000
Kiểm tra thư mục val images: True
Số lượng file trong val images: 1572
Kiểm tra thư mục val labels: True
Số lượng file trong val labels: 1572


In [4]:
from ultralytics import YOLO
import os
import yaml
import torch
import shutil

def train_model_kaggle():
    # Kiểm tra GPU
    device = 0 if torch.cuda.is_available() else 'cpu'
    print(f"Thiết bị: {'GPU' if device == 0 else 'CPU'}")

    # Đường dẫn dữ liệu từ /kaggle/input/
    train_images = '/kaggle/input/drown-datasets/datasets/train/images'
    val_images = '/kaggle/input/drown-datasets/datasets/val/images'

    # Kiểm tra dữ liệu
    for path, name in [(train_images, 'train images'), (val_images, 'val images')]:
        exists = os.path.exists(path)
        print(f"Kiểm tra thư mục {name}: {exists}")
        if exists:
            print(f"Số lượng file trong {name}: {len(os.listdir(path))}")

    # Tạo file YAML
    yaml_path = '/kaggle/working/data.yaml'
    with open(yaml_path, 'w') as f:
        f.write("# YOLOv8 configuration\n")
        f.write(f"train: {train_images}\n")
        f.write(f"val: {val_images}\n")
        f.write("nc: 3\n")
        f.write("names: ['swimming', 'tread water', 'drowning']\n")
    
    print("\nĐã tạo file cấu hình:")
    with open(yaml_path, 'r') as f:
        print(f.read())
    
    # Tải mô hình pretrained
    print("Tải mô hình YOLOv8n...")
    model = YOLO('yolov8n.pt')
    
    # Huấn luyện
    print("\nBắt đầu huấn luyện...")
    results = model.train(
        data=yaml_path,
        epochs=100,
        imgsz=640,
        batch=16,
        name='drowning_detection',
        patience=15,
        save=True,
        save_period=5,
        device=device,
        project='runs',
        exist_ok=True
    )
    
    # Lưu mô hình tốt nhất
    best_model_path = os.path.join('runs', 'detect', 'drowning_detection', 'weights', 'best.pt')
    if os.path.exists(best_model_path):
        saved_best_path = os.path.join('/kaggle/working', 'best_model.pt')
        shutil.copy(best_model_path, saved_best_path)
        print(f"Đã lưu mô hình tốt nhất tại: {saved_best_path}")
    else:
        print(f"Cảnh báo: Không tìm thấy file {best_model_path}")
    
    print("Hoàn thành huấn luyện!")
    return results

if __name__ == "__main__":
    train_model_kaggle()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Thiết bị: GPU
Kiểm tra thư mục train images: True
Số lượng file trong train images: 7000
Kiểm tra thư mục val images: True
Số lượng file trong val images: 1572

Đã tạo file cấu hình:
# YOLOv8 configuration
train: /kaggle/input/drown-datasets/datasets/train/images
val: /kaggle/input/drown-datasets/datasets/val/images
nc: 3
names: ['swimming', 'tread water', 'drowning']

Tải mô hình YOLOv8n...


100%|██████████| 6.25M/6.25M [00:00<00:00, 82.5MB/s]



Bắt đầu huấn luyện...
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=8, project=runs, name=drowning_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


75 epochs completed in 3.966 hours.
Optimizer stripped from runs/drowning_detection/weights/last.pt, 6.2MB
Optimizer stripped from runs/drowning_detection/weights/best.pt, 6.2MB

Validating runs/drowning_detection/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:27<00:00,  1.82it/s]


                   all       1572       2317      0.971      0.978      0.988       0.75
              swimming        991       1381      0.988      0.991      0.995      0.832
           tread water        502        571      0.962      0.971      0.988       0.77
              drowning        353        365      0.962      0.971      0.983      0.649


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/drowning_detection
Cảnh báo: Không tìm thấy file runs/detect/drowning_detection/weights/best.pt
Hoàn thành huấn luyện!
